# first part of spark activity

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

read the json file, and filter the trending and english tweets 

In [2]:
import json
from time import time

t0 = time()
input = sc.textFile("../data/raw/Datasets/Tweets/tweets2.json")
t1 = time()-t0
tweet = input.map(lambda x: json.loads(x))


print("read file took {} seconds".format(round(t1,3)))

read file took 2.469 seconds


In [3]:
# Filter English Tweets
english_tweets = tweet.filter(lambda t: "en" in t["lang"]).map(lambda t:{"id":t["id"],"name":t["user"]["name"],"text":t["text"], "lang":t["lang"]})
trendings= tweet.filter(lambda t: "#" in t["text"][0]).map(lambda t:{"id":t["id"],"name":t["user"]["name"],"text":t["text"], "lang":t["lang"]})
print (trendings.first())
print(english_tweets.first())


    

{'id': 727497507375181824, 'name': 'Carlos González \uf8ff', 'text': '#mecagaandardetraje #findelcomunicado https://t.co/e7wf6vPIw7', 'lang': 'und'}
{'id': 727497507362734080, 'name': 'AmeriGirl', 'text': 'RT @NolteNC: Morning Joe gives huge credit to Hillary for confronting a critic but laughs hysterically when Cruz "shows up" to do same.', 'lang': 'en'}


create the sequence file, in order to save the filtered tweets


In [8]:

import os, shutil

# if the outputs files exist, remove them
if os.path.exists("../reports/sequence_files"):
    shutil.rmtree("../reports/sequence_files")

    
trending_tweet = trendings.map(lambda t: (t["id"], t["text"]))

# Save the trending tweet as sequence file.
trending_tweet.saveAsSequenceFile("../reports/sequence_files")
print (trending_tweet.first())



(727497507375181824, '#mecagaandardetraje #findelcomunicado https://t.co/e7wf6vPIw7')


count words fand using the sequence file(just the filtered data) as the input 
and then save the results into text file 

In [9]:
text_file = sc.textFile("../reports/sequence_files/part-00000")
counts = text_file.flatMap(lambda line: line.split(" ")).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)
counts.saveAsTextFile("../reports/results")
counts.count()



234

read the saved sequence file and then select the first 10 tweets

In [10]:
t0 = time()

seq = sc.sequenceFile("../reports/sequence_files", 
                      "org.apache.hadoop.io.Text", 
                      "org.apache.hadoop.io.IntWritable")

t1 = time()-t0

print("read sequence file took  {} seconds".format(round(t1,3)))

read sequence file took  0.402 seconds


save the first 10 tweets into a text file 

In [11]:
if os.path.exists("../reports/top_10"): 
    shutil.rmtree("../reports/top_10")
    
# Write Tweets into a text file.
top_10 = sc.parallelize(seq.take(10))
top_10.saveAsTextFile("../reports/top_10")
